In [1]:
import os

import tensorflow as tf
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))


I0000 00:00:1761021922.885679   16147 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1761021922.917793   16147 cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1761021924.296525   16147 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


TF: 2.21.0-dev20251017
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


W0000 00:00:1761021925.708708   16147 gpu_device.cc:2456] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0a. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
#importing the necessary packages 

import numpy as np
import pandas as pd
import os 
from PIL import Image
import matplotlib.pyplot as plt

import random
import tensorflow as tf
from keras.models import Sequential
from keras.layers import MaxPooling2D,Conv2D,Dense,Dropout,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

In [4]:
# setting up the data paths

base_dir = '' # base directory  
train_dir = os.path.join(base_dir,'/home/lurpd/Development/Datasets/DistractedDriverSet/imgs/train/')              # train directory

In [5]:
class_def = {
    'c0': 'Safe driving', 
    'c1': 'Distracted',
    'c2': 'Tired', 
}

In [6]:
#add random letterboxing to trainng 
import numpy as np
from PIL import Image

def random_letterbox(img_array):
    #convert numpy array back to PIL for easier manipulation    
    if img_array.ndim == 3 and img_array.shape[-1] == 1:
        img_array = np.squeeze(img_array, axis=-1)

    img = Image.fromarray(np.uint8(img_array))
    target_size = (128, 128)
    
    #random scale factor 
    scale = np.random.uniform(0.8, 1.0)
    
    new_w = int(target_size[0] * scale)
    new_h = int(target_size[1] * scale)
    
    img = img.resize((new_w, new_h), Image.Resampling.LANCZOS)
    
    #black background
    background = Image.new("L", target_size, 0)
    
    #center in the frame
    x_offset = (target_size[0] - new_w) // 2
    y_offset = (target_size[1] - new_h) // 2
    
    background.paste(img, (x_offset, y_offset))
    
    #convert back to numpy float array
    arr = np.array(background).astype(np.float32)
    arr = np.expand_dims(arr, axis=-1)
    return arr / 255.0


In [7]:
image_size = (128,128)  # image shape
batch_size = 64
val_size = 0.2

#data augmentations for better generalizing
train_data_gen = ImageDataGenerator(
    validation_split=0.2,
    preprocessing_function=random_letterbox,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True
)

test_data_gen = ImageDataGenerator()

In [ ]:
# defining the training parameters


train_generator = train_data_gen.flow_from_directory(train_dir,
                                                     target_size = image_size,
                                                     batch_size = batch_size,
                                                     seed=42, 
                                                     shuffle=True,
                                                     subset='training',
                                                     color_mode='grayscale')

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 128, 128, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None, train_generator.num_classes), dtype=tf.float32)
    )
).prefetch(tf.data.AUTOTUNE)

val_generator =  train_data_gen.flow_from_directory(train_dir,
                                               target_size = image_size,
                                               batch_size = batch_size,
                                               seed=42, 
                                               shuffle=True,
                                               subset='validation',
                                               color_mode='grayscale')



Found 31436 images belonging to 3 classes.


W0000 00:00:1761021925.910905   16147 gpu_device.cc:2456] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0a. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


Found 7857 images belonging to 3 classes.


I0000 00:00:1761021926.008928   16147 gpu_device.cc:2040] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9129 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 5070, pci bus id: 0000:01:00.0, compute capability: 12.0a


In [9]:
import collections

class_counts = collections.Counter(train_generator.classes)
for cls, count in class_counts.items():
    print(f"{cls} ({list(train_generator.class_indices.keys())[cls]}): {count}")

AttributeError: '_PrefetchDataset' object has no attribute 'classes'

In [ ]:
for data_batch,label_batch in train_generator:
    print(data_batch.shape)   # train batch
    print(label_batch.shape)  # label batch
    break

batch_x, batch_y = next(train_generator)
print(batch_x.shape, batch_x.min(), batch_x.max())
plt.imshow(batch_x[0].squeeze(), cmap='gray')
plt.show()

In [ ]:
import tensorflow as tf
from keras.layers import Dropout

class CPUDropout(Dropout):
    def call(self, inputs, training=None):
        with tf.device('/CPU:0'):
            return super().call(inputs, training=training)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
import tensorflow as tf

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,1)),
    MaxPooling2D(2,2),
    CPUDropout(0.3),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    CPUDropout(0.3),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    CPUDropout(0.4),

    Flatten(),
    Dense(256, activation='relu'),
    CPUDropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])


model.summary()


In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
#autocalculate class weight

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

import collections

class_counts = collections.Counter(train_generator.classes)

print("\nImage count per class:")
for class_index, count in sorted(class_counts.items()):
    class_name = list(train_generator.class_indices.keys())[class_index]
    print(f"  {class_index} ({class_name}): {count} images")

class_indices = train_generator.class_indices
num_classes = len(class_indices)
class_labels = np.unique(train_generator.classes)

weights = compute_class_weight(
    class_weight="balanced",
    classes=class_labels,
    y=train_generator.classes
)

class_weights = dict(enumerate(weights))

#print to make sure
print("\nAuto-calculated class weights:")
for i, w in class_weights.items():
    class_name = list(class_indices.keys())[i]
    print(f"  {i} ({class_name}): {w:.3f}")

# manual weights just in case
class_weights = {
    0: 1.2,  #Safe driving
    1: 0.5,  #distracted
    2: 0.0,  #tired
}

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
]

history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_generator,
    class_weight=class_weights,
    # callbacks=callbacks
)

In [ ]:
model.save("distracted_driver_detection.keras")

In [ ]:
tr_loss = history.history['loss']
tr_accuracy = history.history['accuracy']

val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

In [ ]:
epchs = list(range(1,len(tr_loss)+1))
plt.plot(epchs,tr_loss,label='Train')
plt.plot(epchs,val_loss,label='Test')
plt.title("Training and Validation loss")
plt.legend()
plt.show()


plt.plot(epchs,tr_accuracy,label='Train')
plt.plot(epchs,val_accuracy,label='Test')
plt.title("Training and Validation accuracy")
plt.legend()
plt.show()

In [ ]:
# test model with real data
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import os
import math

# load model
model = load_model("distracted_driver_detection.keras",
                   custom_objects={"CPUDropout": Dropout})

# class names
class_names = ["Safe driving", "Distracted", "Tired"]

# folder
folder = "/home/lurpd/Development/Datasets/MyData/"
target_size = (128, 128)

image_paths = []
for i in range(30, 63):
    for ext in [".png", ".PNG", ".jpg", ".JPG", ".jpeg", ".JPEG"]:
        candidate = os.path.join(folder, f"{i}{ext}")
        if os.path.exists(candidate):
            image_paths.append(candidate)
            break  

# display grid
cols = 8
rows = math.ceil(len(image_paths) / cols)
max_width_px = 2500
max_height_px = 1300
dpi = 100
fig_width = min(max_width_px / dpi, cols * 3)
fig_height = min(max_height_px / dpi, rows * 3)
plt.figure(figsize=(fig_width, fig_height), dpi=dpi)

# process/display each image
for idx, img_path in enumerate(image_paths):
    img = Image.open(img_path).convert("L")
    img = ImageOps.pad(img, target_size, color=0, method=Image.Resampling.LANCZOS)
    img_array = np.expand_dims(np.array(img).astype(np.float32) / 255.0, axis=(0, -1))

    # predict
    predictions = model.predict(img_array, verbose=0)
    predicted_class = np.argmax(predictions, axis=1)[0]
    predicted_label = class_names[predicted_class]
    confidence = np.max(predictions) * 100

    # display
    plt.subplot(rows, cols, idx + 1)
    plt.imshow(img, cmap="gray")
    plt.axis("off")
    plt.text(
        0.5, -0.05,
        f"{os.path.basename(img_path)}\n{predicted_label} ({confidence:.1f}%)",
        fontsize=9,
        ha="center", va="top",
        transform=plt.gca().transAxes
    )

plt.tight_layout()
plt.draw()

# save automatically
output_path = "predictions_grid.png"
plt.savefig(output_path, dpi=150, bbox_inches="tight")
print(f"✅ Saved predictions grid to {output_path}")

# open automatically (works in WSL or native Linux)
try:
    if os.path.exists("/mnt/c/Windows"):
        os.system(f"explorer.exe {output_path.replace('/', '\\\\')}")
    else:
        os.system(f"xdg-open {output_path}")
except Exception as e:
    print(f"⚠️ Could not auto-open image: {e}")
